# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [17]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
g_key = ""

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [18]:
#raw_df.to_csv("WeatherPy.csv")
raw_df=pd.read_csv("WeatherPy.csv")

raw_df.describe()

,Unnamed: 0,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,5.690000e+02
mean,284.000000,21.591186,15.129421,49.660896,72.165202,47.008787,8.116801,1.614055e+09
std,164.400426,32.790580,90.937531,32.912307,22.023534,39.409840,5.577580,1.762774e+02
min,0.000000,-54.800000,-175.200000,-34.600000,1.000000,0.000000,0.160000,1.614054e+09
25%,142.000000,-6.433300,-65.102300,33.800000,63.000000,1.000000,4.000000,1.614055e+09
50%,284.000000,27.976900,20.441700,59.000000,78.000000,40.000000,6.910000,1.614055e+09
75%,426.000000,48.568200,91.390300,75.200000,88.000000,90.000000,10.380000,1.614055e+09
max,568.000000,78.218600,179.316700,105.800000,100.000000,100.000000,34.520000,1.614055e+09


In [19]:
#del raw_df["Unnamed: 0"]
raw_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,ponta do sol,32.6667,-17.1000,60.01,84,36,1.01,PT,1614054571
1,morgan city,29.6994,-91.2068,50.00,87,1,6.93,US,1614054571
2,duku,11.2361,4.9073,70.93,11,0,7.87,NG,1614054572
3,caborca,30.5838,-111.1010,64.99,17,79,5.01,MX,1614054573
4,punta arenas,-53.1500,-70.9167,44.60,57,75,6.91,CL,1614054470


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [20]:
#Configure gmaps
gmaps.configure(api_key=g_key)

In [21]:
#Store locations
locations=raw_df[["Lat","Lng"]]

#Store Weight
weight=raw_df["Humidity"]

In [27]:
#Plot Heatmap 
fig = gmaps.figure(center=(46,-46),zoom_level=2.2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=300,
                                 point_radius=5)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [33]:
ideal=raw_df.loc[(raw_df["Max Temp"]<80)&(raw_df["Max Temp"]>70)&(raw_df["Wind Speed"]<10)&(raw_df["Cloudiness"]==0)]
ideal
#no rows to drop; all rows meet condition

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,duku,11.2361,4.9073,70.93,11,0,7.87,NG,1614054572
46,bubaque,11.2833,-15.8333,76.44,67,0,7.07,GW,1614054617
59,sao filipe,14.8961,-24.4956,70.54,76,0,8.21,CV,1614054634
199,cidreira,-30.1811,-50.2056,73.58,86,0,6.71,BR,1614054768
256,janakpur,26.7183,85.9065,79.68,27,0,5.55,NP,1614054831
302,dwarka,22.2394,68.9678,79.45,47,0,8.19,IN,1614054886
329,sadat,25.6833,83.2833,71.60,53,0,4.61,IN,1614054900
362,paicandu,-23.4575,-52.0486,78.80,61,0,4.61,BR,1614054945
428,haimen,23.1935,116.6142,75.20,46,0,8.95,CN,1614055006
457,iguala,18.3438,-99.5372,72.75,35,0,3.40,MX,1614055020


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [35]:
hotel_df=ideal

#Add Column Name
hotel_df["Hotel Name"]=""
hotel_df.head()

<ipython-input-35-0e094c959aa0>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"]=""


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
2,duku,11.2361,4.9073,70.93,11,0,7.87,NG,1614054572,
46,bubaque,11.2833,-15.8333,76.44,67,0,7.07,GW,1614054617,
59,sao filipe,14.8961,-24.4956,70.54,76,0,8.21,CV,1614054634,
199,cidreira,-30.1811,-50.2056,73.58,86,0,6.71,BR,1614054768,
256,janakpur,26.7183,85.9065,79.68,27,0,5.55,NP,1614054831,


In [36]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "type": "hotels",
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get lat and lng from dataframe
    lat = row['Lat']
    lng = row['Lng']
   
    # add keyword to params dict
    params['location'] = f'{lat},{lng}'

    # assemble url and make API request
    #print(f"Retrieving Results for Index {index}: {restr_type}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:

        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        print(f"Closest hotel is {results[0]['name']}.")
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

C:\Users\njeli\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


Closest hotel is Dukku.
------------
Closest hotel is Bubaque.
------------
Closest hotel is Sao Filipe.
------------
Closest hotel is CIDREIRA.
------------
Closest hotel is Janakpur.
------------
Closest hotel is Dwarka.
------------
Closest hotel is Bahariyabad.
------------
Closest hotel is Maringá.
------------
Closest hotel is Shantou.
------------
Closest hotel is Iguala.
------------
Closest hotel is Cienfuegos.
------------
Closest hotel is Maceió.
------------
Closest hotel is Chabahar.
------------
Closest hotel is Turbat.
------------
Closest hotel is Alligator Pond.
------------


In [38]:
hotel_df=hotel_df.dropna()
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
2,duku,11.2361,4.9073,70.93,11,0,7.87,NG,1614054572,Dukku
46,bubaque,11.2833,-15.8333,76.44,67,0,7.07,GW,1614054617,Bubaque
59,sao filipe,14.8961,-24.4956,70.54,76,0,8.21,CV,1614054634,Sao Filipe
199,cidreira,-30.1811,-50.2056,73.58,86,0,6.71,BR,1614054768,CIDREIRA
256,janakpur,26.7183,85.9065,79.68,27,0,5.55,NP,1614054831,Janakpur
302,dwarka,22.2394,68.9678,79.45,47,0,8.19,IN,1614054886,Dwarka
329,sadat,25.6833,83.2833,71.60,53,0,4.61,IN,1614054900,Bahariyabad
362,paicandu,-23.4575,-52.0486,78.80,61,0,4.61,BR,1614054945,Maringá
428,haimen,23.1935,116.6142,75.20,46,0,8.95,CN,1614055006,Shantou
457,iguala,18.3438,-99.5372,72.75,35,0,3.40,MX,1614055020,Iguala


In [41]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [42]:
# Add marker layer ontop of heat map
#Plot Heatmap 

# Create heat layer
marker = gmaps.marker_layer(locations,info_box_content=hotel_info)
                                 

# Add layer
fig.add_layer(marker)

# Display figure
fig




Figure(layout=FigureLayout(height='420px'))